In [ ]:
lammps_input_script = """# Sample LAMMPS input script for viscosity of 2d LJ liquid
# NEMD via fix deform and fix nvt/sllod

# Settings
variable	    x equal 20
variable	    y equal 20

variable	    rho equal 0.6
variable        t equal 1.0
variable	    rc equal 2.5

variable	    srate equal 2.7 # velocity of top edge

# Problem setup
units		    lj
dimension	    2
atom_style	    atomic
neigh_modify	delay 0 every 1

lattice         sq2 ${rho}
region          simbox prism 0 $x 0 $y -0.1 0.1 0.0 0.0 0.0
create_box      1 simbox
create_atoms    1 box

pair_style      lj/cut ${rc}
pair_coeff      * * 1 1

mass            * 1.0

# 1. Equilibration run
velocity        all create $t 97287
fix             1 all nve
fix	            2 all langevin $t $t 0.1 498094
fix	            3 all enforce2d

thermo          1000
run	            5000

unfix  	        1
unfix		    2

# 2. Turn on NEMD shear and equilibrate some more
velocity	    all scale $t

# Shear rate defined relative to perpendicular dimension
variable	    xyrate equal ${srate}/ly 

fix		        1 all nvt/sllod temp $t $t 0.1

# Perform deformation every 1 timestep
# erate - engineering shear strain rate (1/time units)
fix		        2 all deform 1 xy erate ${xyrate} remap v

# The average in each chunk is computed every 5000 timesteps using 250 samples taken at intervals of 
# 20 timesteps from the preceding timesteps.
# For this case, there seems to be no difference between lower, center, and upper options.
compute         layers_center all chunk/atom bin/1d y center 0.05 units reduced
fix		        4 all ave/chunk 20 250 5000 layers_center vx file profile_center.nemd.2d

compute		    usual all temp
compute		    tilt all temp/deform

thermo          1000
thermo_style	custom step temp c_usual epair etotal press pxy
thermo_modify	temp tilt

run		        50000

# 3. Data gathering run
# This only takes into account the top and bottom velocity points to fit the linear profile
# It doesn't even use fix 4, which is the only one that actually computes the profile
# The viscosity is calculated from the slope of the linear profile
variable        timestep equal step
variable	    visc equal -pxy/(v_srate/ly)

# The average in each chunk is computed every 1000 timesteps using 100 samples taken at intervals of 
# 10 timesteps from the preceding timesteps.
# The running average sums the average the average values computed every 1000 timesteps
fix             vprint all print 10 "${timestep} ${visc}" file viscosity_output.txt screen no
fix		        vave all ave/time 10 100 1000 v_visc ave running start 56000 file viscosity_averages.txt

thermo          1000
thermo_style	custom step temp press pxy ly v_visc f_vave
thermo_modify	temp tilt

# only need to run for 5400 steps to make a good 100-frame movie
# set 54K steps above in equil and 5400 here simply to make good movie
# 54K and 5400 are multiples of box-swap periodicity = 2700 steps

dump	        1 all custom 50 dump.nemd.2d id type x y z vx vy vz

run		        50000
"""

In [50]:
# Write the content to a file called in.nemd.2d
with open("in.nemd.2d", "w") as file:
    file.write(lammps_input_script)

In [2]:
# Function to plot data for a specific timestep using Plotly
def plot_timestep(timestep):
    if timestep in chunk_data:
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=chunk_data[timestep]['vx'],
            y=chunk_data[timestep]['coord1'],
            mode='lines+markers',
            name=f'Timestep {timestep}'
        ))
        fig.update_layout(
            title=f'Velocity Profile at Timestep {timestep}',
            yaxis_title='Coord1',
            xaxis_title='vx',
            template='plotly_white'
        )
        fig.show()
    else:
        print(f"No data available for timestep {timestep}")

In [3]:
import plotly.graph_objects as go

# Initialize dictionaries to store the data for each chunk
chunk_data = {}

# Read the file and parse the data
with open('profile_center.nemd.2d', 'r') as file:
    lines = file.readlines()
    for line in lines:
        if line.startswith('#') or line.strip() == '':
            continue
        parts = line.split()
        if len(parts) == 3:
            # This is a timestep line
            current_timestep = int(parts[0])
        elif len(parts) == 4:
            # This is a data line
            chunk = int(parts[0])
            coord = float(parts[1])
            ncount = float(parts[2])
            vx = float(parts[3])
            
            if current_timestep not in chunk_data:
                chunk_data[current_timestep] = {'chunk': [], 'coord1': [], 'ncounts': [], 'vx': []}
                
            chunk_data[current_timestep]['chunk'].append(chunk)
            chunk_data[current_timestep]['coord1'].append(coord)
            chunk_data[current_timestep]['ncounts'].append(ncount)
            chunk_data[current_timestep]['vx'].append(vx)

In [12]:
plot_timestep(15000)

In [46]:
import numpy as np

# Read viscosity_output.txt into numpy array ignoring the first line
data = np.loadtxt('viscosity_output.txt', skiprows=1)

# Reshape the second column to group every 100 values
second_column = data[:, 1][1:]
reshaped = second_column[:].reshape(-1, 100)

# Compute the mean for each group
calculated_averages = reshaped.mean(axis=1)

# Compute the running average
calculated_running_averages = np.cumsum(calculated_averages) / np.arange(1, len(calculated_averages) + 1)

In [44]:
# Read the viscosity_averages.txt file into a numpy array ignoring the first two lines
averages = np.loadtxt('viscosity_averages.txt', skiprows=2)
running_averages = averages[:, 1]

In [48]:
# Compare the LAMMPS calculated running average with the one we computed
np.allclose(running_averages, calculated_running_averages, atol=1e-6)

True